In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# confirm that the "epa" database is in MongoDB
print(mongo.list_database_names())

['admin', 'autosaurus', 'classDB', 'config', 'epa', 'fruits_db', 'local']


In [4]:
# assign the database to a variable name
db = mongo['epa']

In [5]:
# review the collections in our database
print(db.list_collection_names())

['ohio_daily_records', 'annual_aqi_by_county', 'ohio_air']


In [6]:
# assign each collection to a variable
ohio_daily_records = db['ohio_daily_records']
ohio_air = db['ohio_air']
annual_aqi_by_county = db['annual_aqi_by_county']

In [7]:
# Display the total number of documents in the annual_aqi_by_county collection
annual_aqi_by_county.count_documents({})

647

In [8]:
# Create a query that finds the documents that have a "parameter" of "Sulfur dioxide" in the ohio_air collection
query = {'parameter': "Sulfur dioxide"}

# Print the number of results
print("Number of documents in result:", ohio_air.count_documents(query))

Number of documents in result: 6764


In [9]:
# Pretty print just the first result from the Sulfur dioxide query using list indexing
print("First result:")
results = ohio_air.find(query)
pprint(results[0])

First result:
{'_id': ObjectId('634d7e716478c3fa5a3e4d1d'),
 'aqi': 0,
 'arithmetic_mean': 0.0,
 'cbsa': 'Cleveland-Elyria, OH',
 'cbsa_code': '17460',
 'city': 'Newburgh Heights',
 'county': 'Cuyahoga',
 'county_code': '035',
 'date_local': '2022-01-01',
 'date_of_last_change': '2022-09-19',
 'datum': 'NAD83',
 'event_type': 'No Events',
 'first_max_hour': 0,
 'first_max_value': 0.0,
 'latitude': 41.446624,
 'local_site_name': 'Harvard Yards',
 'longitude': -81.662356,
 'method': 'Instrumental - Ecotech Serinus 50',
 'method_code': '188',
 'observation_count': 24,
 'observation_percent': 100.0,
 'parameter': 'Sulfur dioxide',
 'parameter_code': '42401',
 'poc': 1,
 'pollutant_standard': 'SO2 Annual 1971',
 'sample_duration': '1 HOUR',
 'sample_duration_code': '1',
 'site_address': '4600 HARVARD AVE.',
 'site_number': '0065',
 'state': 'Ohio',
 'state_code': '39',
 'units_of_measure': 'Parts per billion',
 'validity_indicator': 'Y'}


In [10]:
# Select only the 'parameter', 'units_of_measure', 'observation_count', 'date_local', 'local_site_name', 
# 'site_address', 'city', and 'county' fields from the ohio_air collection
query = {}
fields = {'parameter': 1, 'units_of_measure': 1, 'observation_count': 1, 'date_local': 1, 
         'local_site_name': 1, 'site_address': 1, 'city': 1, 'county': 1}

# Capture the results to a variable
results = ohio_air.find(query, fields)

# Pretty print the first two results
for i in range(2):
    pprint(results[i])

{'_id': ObjectId('634d7e716478c3fa5a3e4d1d'),
 'city': 'Newburgh Heights',
 'county': 'Cuyahoga',
 'date_local': '2022-01-01',
 'local_site_name': 'Harvard Yards',
 'observation_count': 24,
 'parameter': 'Sulfur dioxide',
 'site_address': '4600 HARVARD AVE.',
 'units_of_measure': 'Parts per billion'}
{'_id': ObjectId('634d7e716478c3fa5a3e4d1e'),
 'city': 'Newburgh Heights',
 'county': 'Cuyahoga',
 'date_local': '2022-01-02',
 'local_site_name': 'Harvard Yards',
 'observation_count': 24,
 'parameter': 'Sulfur dioxide',
 'site_address': '4600 HARVARD AVE.',
 'units_of_measure': 'Parts per billion'}


In [11]:
# Select every field from the ohio_daily_records collection except the "COUNTY_CODE" and "STATE_CODE" fields
query = {}
fields = {'COUNTY_CODE': 0, 'STATE_CODE': 0}

# Capture the results to a variable
results = ohio_daily_records.find(query, fields)

# Pretty print the first two results
for i in range(2):
    pprint(results[i])

{'CBSA_CODE': 17460.0,
 'CBSA_NAME': 'Cleveland-Elyria, OH',
 'CO': {'AQS_PARAMETER_CODE': '42101',
        'AQS_PARAMETER_DESC': 'Carbon monoxide',
        'DAILY_AQI_VALUE': '1',
        'DAILY_OBS_COUNT': 5,
        'Daily Max 8-hour CO Concentration': 0.1,
        'PERCENT_COMPLETE': '21.0',
        'UNITS': 'ppm'},
 'COUNTY': 'Cuyahoga',
 'Date': '01/03/2022',
 'NO2': None,
 'POC': 1,
 'SITE_LATITUDE': 41.504605,
 'SITE_LONGITUDE': -81.69045,
 'SO2': None,
 'STATE': 'Ohio',
 'Site ID': 390350051,
 'Site Name': 'Galleria',
 'Source': 'AQS',
 '_id': ObjectId('634d7e65004ec0bf1deb3a59')}
{'CBSA_CODE': 17460.0,
 'CBSA_NAME': 'Cleveland-Elyria, OH',
 'CO': {'AQS_PARAMETER_CODE': '42101',
        'AQS_PARAMETER_DESC': 'Carbon monoxide',
        'DAILY_AQI_VALUE': '1',
        'DAILY_OBS_COUNT': 5,
        'Daily Max 8-hour CO Concentration': 0.1,
        'PERCENT_COMPLETE': '21.0',
        'UNITS': 'ppm'},
 'COUNTY': 'Cuyahoga',
 'Date': '01/11/2022',
 'NO2': None,
 'POC': 1,
 'SITE_LAT

In [12]:
# Change the data type from String to Double for CO.PERCENT_COMPLETE 
# and from String to Int for CO.DAILY_AQI_VALUE
# in the ohio_daily_records collection
ohio_daily_records.update_many({}, [ {'$set': { "CO.PERCENT_COMPLETE" : {'$toDouble': "$CO.PERCENT_COMPLETE"},
                                                "CO.DAILY_AQI_VALUE" : {'$toInt': "$CO.DAILY_AQI_VALUE"}
                                              } 
                                     } ]
                              )

In [14]:
# Create a query that finds the documents in the ohio_daily_records collection 
# where CO.UNITS matches "ppm" and NO2.UNITS matches "ppb", and 
# select only the following fields: "CBSA_NAME", "COUNTY", "Site Name", "Date", "CO", "NO2", and "SO2". 
query = {'CO.UNITS': "ppm", 'NO2.UNITS': "ppb"}
fields = {'CBSA_NAME': 1, 'COUNTY': 1, 'Site Name': 1, 'Date': 1,
          'CO': 1, 'NO2': 1, 'SO2': 1}

# Capture the results to a variable
results = ohio_daily_records.find(query, fields)

# Pretty print the first two results
for i in range(2):
    pprint(results[i])

{'CBSA_NAME': 'Cleveland-Elyria, OH',
 'CO': {'AQS_PARAMETER_CODE': '42101',
        'AQS_PARAMETER_DESC': 'Carbon monoxide',
        'DAILY_AQI_VALUE': 3,
        'DAILY_OBS_COUNT': 18,
        'Daily Max 8-hour CO Concentration': 0.3,
        'PERCENT_COMPLETE': 75.0,
        'UNITS': 'ppm'},
 'COUNTY': 'Cuyahoga',
 'Date': '01/01/2022',
 'NO2': {'AQS_PARAMETER_CODE': '42602',
         'AQS_PARAMETER_DESC': 'Nitrogen dioxide (NO2)',
         'DAILY_AQI_VALUE': 12,
         'DAILY_OBS_COUNT': 23,
         'Daily Max 1-hour NO2 Concentration': 13.0,
         'PERCENT_COMPLETE': 96.0,
         'UNITS': 'ppb'},
 'SO2': {'AQS_PARAMETER_CODE': '42401',
         'AQS_PARAMETER_DESC': 'Sulfur dioxide',
         'DAILY_AQI_VALUE': 0,
         'DAILY_OBS_COUNT': 23,
         'Daily Max 1-hour SO2 Concentration': 0.0,
         'PERCENT_COMPLETE': 96.0,
         'UNITS': 'ppb'},
 'Site Name': 'GT Craig NCore',
 '_id': ObjectId('634d7e65004ec0bf1deb3b16')}
{'CBSA_NAME': 'Cleveland-Elyria, OH',
 '

In [15]:
# Create a query that finds the documents where the "State" is "Ohio" in the annual_aqi_by_county collection 
# and returns only the "County," "State," "Days with AQI," and "Max AQI" fields
query = {'State': "Ohio"}
fields = {'County': 1, 'State': 1, 'Days with AQI': 1, 'Max AQI': 1}

# Capture the results to a variable
results = annual_aqi_by_county.find(query, fields)

# Pretty print the first four results
for i in range(4):
    pprint(results[i])

{'County': 'Ashtabula',
 'Days with AQI': 26,
 'Max AQI': 77,
 'State': 'Ohio',
 '_id': ObjectId('634d7e60e987b047d7007e55')}
{'County': 'Allen',
 'Days with AQI': 90,
 'Max AQI': 93,
 'State': 'Ohio',
 '_id': ObjectId('634d7e60e987b047d7007e56')}
{'County': 'Athens',
 'Days with AQI': 29,
 'Max AQI': 52,
 'State': 'Ohio',
 '_id': ObjectId('634d7e60e987b047d7007e57')}
{'County': 'Belmont',
 'Days with AQI': 89,
 'Max AQI': 60,
 'State': 'Ohio',
 '_id': ObjectId('634d7e60e987b047d7007e58')}
